In [ ]:
# Разбила на две части, т.к. почему-то лагало

### 1. Импорт библиотек

In [ ]:
import pandas as pd
import numpy as np

import seaborn as sns
# import plotly.express as px
# import plotly.graph_objects as go
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

from sklearn.decomposition import TruncatedSVD
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest,chi2,RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, silhouette_score
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.cluster import KMeans

from scipy.sparse import csr_matrix

from pylab import rcParams

import datetime as dt

import warnings
warnings.filterwarnings('ignore')

import math, random
from collections import defaultdict, Counter

import datetime as dt

%matplotlib inline

# графики в svg выглядят более четкими
%config InlineBackend.figure_format = 'svg'

import warnings
warnings.filterwarnings('ignore')

# увеличим дефолтный размер графиков
from pylab import rcParams

rcParams["figure.figsize"] = 10, 6
sns.set_style("whitegrid", {'axes.grid' : True})

### 2. Кластеры

In [ ]:
# Берём тут подготовленный датасет из первой части
df = pd.read_csv('df.csv', sep='\t')
df.head()

In [ ]:
# Ещё раз: значимые данные здесь только сумма и дата\время, остальное либо айдишники, либо константные значения

In [ ]:
scaling = StandardScaler()

scaling.fit(df[['Сумма', 'Месяц', 'Час']]) 
scaled_df=scaling.transform(df[['Сумма', 'Месяц', 'Час']])

scaled_df = pd.DataFrame(scaled_df)
scaled_df.columns = ['Сумма', 'Месяц', 'Час']

In [ ]:
wcss = []
silhouette_scr = [] 

# Убираем выбросы (несколько значений, где сумма сильно больше)
x = scaled_df[scaled_df['Сумма']<=20]

for i in range(2, 11):
    kmeans = KMeans(n_clusters=i, init='k-means++', random_state=42)
    preds = kmeans.fit_predict(x)
    wcss.append(kmeans.inertia_)
    score = silhouette_score(x, preds)
    silhouette_scr.append(score)
    
plt.subplot(1, 2, 1)
plt.plot(range(2, 11), wcss, marker='o')
plt.title('Метод локтя')

plt.subplot(1, 2, 2)
plt.plot(range(2, 11), silhouette_scr, marker='o')
plt.title('Метод силуэта')

plt.show() 

In [ ]:
# Согласно метрикам, более-менее оптимальное количество кластеров равно 6

In [ ]:
#features = X_train.columns

kmeans = KMeans(n_clusters=6, init='k-means++', random_state=42)
kmeans.fit(x)
segment_kmeans = kmeans.fit_predict(x)

df_clusters = pd.DataFrame(x)

df_clusters['segment_kmeans'] = segment_kmeans
df_clusters['segment_kmeans'].value_counts() #посмотрим размеры получившихся кластеров

In [ ]:
sns.set(rc={'figure.figsize':(10,5)}, style='ticks')

fig, ax = plt.subplots(1,2)


ax[0].set_xlabel('Компонента 0')
ax[0].set_ylabel('Компонента 1')
sns.scatterplot(
                x=df_clusters['Сумма'], y=df_clusters["Месяц"], 
                hue=df_clusters['segment_kmeans'],ax=ax[0])

ax[0].set_xlabel('Компонента 2')
ax[0].set_ylabel('Компонента 1')

sns.scatterplot(
                x=df_clusters["Сумма"], y=df_clusters["Час"], 
                hue=df_clusters['segment_kmeans'],  ax=ax[1])
fig.show()

In [ ]:
# Попробуем проверисти кластеризацию с помощью DBSCAN

In [ ]:
from sklearn.cluster import DBSCAN
from collections import Counter
from sklearn.neighbors import NearestNeighbors

In [ ]:
min_pts = 50
nn = NearestNeighbors(n_neighbors = min_pts, metric='cosine')
nn.fit(x)

dist, _ = nn.kneighbors(x)

In [ ]:
dist_last = dist[:, -1]
dist_last = np.sort(dist_last)

plt.figure(figsize = (8,5))
plt.plot(dist_last[:-1000]) # Последние 9000 сильно большие, их отрезаем, чтобы на графичке был лучше виден изгиб
# (Важно найти такое расстояние (ось у), при котором график только начинает возрастать)

In [ ]:
# По графику можем определить расстояние, на котором находятся близкие соседи (до изгиба)
# Примем его за окрестность 
eps = 0.01

clustering = DBSCAN(eps = eps, min_samples=min_pts, metric='cosine', n_jobs=-1).fit_predict(x)

In [ ]:
df_clusters['segment_dbscan'] = clustering
df_clusters['segment_dbscan'].value_counts() 

In [ ]:
sns.set(rc={'figure.figsize':(10,5)}, style='ticks')

fig, ax = plt.subplots(1,2)


ax[0].set_xlabel('Компонента 0')
ax[0].set_ylabel('Компонента 1')
sns.scatterplot(
                x=df_clusters['Сумма'], y=df_clusters["Месяц"], 
                hue=df_clusters['segment_dbscan'],ax=ax[0])

ax[0].set_xlabel('Компонента 2')
ax[0].set_ylabel('Компонента 1')

sns.scatterplot(
                x=df_clusters["Сумма"], y=df_clusters["Час"], 
                hue=df_clusters['segment_dbscan'],  ax=ax[1])
fig.show()

### 7. Важные признаки

In [ ]:
# Таргет Сумма - непрерывное значение. Для оценки признаков нужны классы, поэтому сведем цену к нескольким диапазонам

def bucketer(y):
    if y < 1000:
        return 0
    if y < 3000:
        return 1
    if y < 5000:
        return 2
    if y < 7000:
        return 3
    if y < 9000:
        return 4
    if y < 11000:
        return 5
    if y < 13000:
        return 6
    if y < 15000:
        return 7
    if y < 17000:
        return 8
    return 9
    
y = df['Сумма'].apply(lambda x: bucketer(x))
y.value_counts()

In [ ]:
#Критерий согласия Хи-квадрат Пирсона

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

x = df[['Месяц', 'Час']]

fit = SelectKBest(score_func = chi2, k = 'all').fit(x, y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(x.columns)

featureScores = pd.concat([dfcolumns, dfscores], axis = 1)
featureScores.columns = ['Specs', 'Score']  
featureScores

In [ ]:
# Заодно оценим важность признаков с помощью леса
# (когда всего два признака, результат неинтересный, но почему бы и нет)

In [ ]:
param_grid = { 
    'n_estimators': [10, 20, 50],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,5,6,8],
    'criterion' :['gini', 'entropy']
}

In [ ]:
model = RandomForestClassifier(random_state = 42)

In [ ]:
from sklearn.model_selection import GridSearchCV

CV_rfc = GridSearchCV(estimator=model, param_grid=param_grid, cv= 5)
CV_rfc.fit(x, y)

In [ ]:
CV_rfc.best_params_  # Подобрали оптимальные параметры

In [ ]:
clf = RandomForestClassifier(random_state = 42, 
                               criterion = 'entropy',
                               max_depth=6,
                               max_features='auto',
                               n_estimators=10)
clf.fit(x, y)

In [ ]:
plt.figure(figsize=(3,3))
plt.bar(['Месяц', 'Час'], clf.feature_importances_)
plt.xticks(rotation=45)

### 8.	Рекомендации

In [ ]:
#чтобы дать рекомендации, нам нужно будет вычислить любой индекс сходства вне косинусного сходства, 
#евклидово расстояние, расстояние Жаккара и т. д. Здесь мы собираемся использовать косинусное сходство.

from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Список товаров
item_list = list(df['Товар'].unique())
len(item_list)

In [ ]:
# Список покупателей
customer_list = list(df['Клиент'].unique())
len(customer_list)

In [ ]:
# Считаем матрицу взаимодействия (строки - товары, столбцы - покупатели, на пересечении - 
# число раз, которое покупатель приобрел данных товар)

item_customer_matrix = []
for item in item_list:
    item_customers_row = list(np.zeros(len(customer_list))) # Пустая строча покупателей, сначала нули
    
    item_customers = df[df['Товар'] == item]['Клиент'].values # Список покупателей для текущего товара (с повторениями)
    for item_customer in item_customers: # Для каждой записи о покупке
        item_customer_index = customer_list.index(item_customer)
        item_customers_row[item_customer_index] += 1
        
    item_customer_matrix.append(item_customers_row)
    
item_customer_matrix = np.array(item_customer_matrix)    

In [ ]:
item_customer_matrix.shape

In [ ]:
from scipy import spatial

# Поиск к ближайших соседей
def get_similar_items(item_row, item_customer_matrix, k = 10):

    item_item_similarity = np.array([])
    for i in item_customer_matrix:
        cos_distance = spatial.distance.cosine(item_row, i)
        item_item_similarity = np.append(item_item_similarity, cos_distance)   
        
    nearest_items = []
    while len(item_item_similarity) > 0 | len(nearest_items) < k:
        ind = item_item_similarity.argmax()
        el = item_list[ind]
        item_item_similarity = np.delete(item_item_similarity, ind)
        
        if el not in nearest_items:
            nearest_items.append(el)
    return nearest_items[1:] # ближайший сосед - это сам элемент, его брать не будем


def one_item_based_recommendations(item_number, item_customer_matrix, k = 10):
    try:
        item_id = item_list.index(item_number)
    except:
        return [] 
    items_to_recommend = get_similar_items(item_customer_matrix[item_id], item_customer_matrix, k + 1)
    return items_to_recommend

In [ ]:
# Поскольку в нашем датасете не особо много информации о товарах, зато есть история покупок, оптимальным вариантов будет
# коллаборативная фильтрация

# Коллаборативная фильтрация, основанная на товарах: 
# Берем все товары, купленные пользователем, ищем для них похожие
# Можно рекомендовать те, что встретились чаще всего, или ориентироваться на дату - например, смотреть по самым последним
# покупкам.
# У нас будет сразу оба варианта)

def item_based_recommendations(customer_number, item_customer_matrix, k = 10, by_time = False):
    
    customer_items = list(df[df['Клиент']==customer_number].sort_values(by=['Дата'])['Товар'].values)
    items_to_recommend = []
    
    for customer_item in customer_items:
        similar_items = one_item_based_recommendations(customer_item, item_customer_matrix, k)
        items_to_recommend += similar_items 
        
        if by_time is True:
            if len(set(items_to_recommend)) >= k:
                break
        
    return [i[0] for i in Counter(items_to_recommend).most_common(k)]

In [ ]:
# Коллаборативная фильтрация по пользователям:
# ищем пользователей с похожей историей покупок и рекомендуем то, что они покупали

def customer_based_recommendations(customer_number, item_customer_matrix, k = 10):
    try:
        customer_id = customer_list.index(customer_number)
    except:
        return []   
    
    similar_customers = get_similar_items(item_customer_matrix[:,customer_id], 
                                          item_customer_matrix.transpose())
    
    items_to_recommend = []
    for similar_customer in similar_customers: 
        items_to_recommend += list(df[df['Клиент'] == similar_customer]['Товар'].values)   
    return [i[0] for i in Counter(items_to_recommend).most_common(k)]

In [ ]:
# Три вида рекомендаций для пользователя с id = 27

items_by_customer = customer_based_recommendations(27, item_customer_matrix, k = 10)
items_by_item_last = item_based_recommendations(27, item_customer_matrix, k = 10, by_time = False)
items_by_item_last_period = item_based_recommendations(27, item_customer_matrix, k = 10, by_time = True)

In [ ]:
print(items_by_customer)
print(items_by_item_last)
print(items_by_item_last_period)

## Вывод